In [7]:
import pandas as pd
pd.set_option("display.float_format", "{:.2f}".format)

from scipy.io import arff

# Cargar el archivo .arff
data, meta = arff.loadarff(r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\Github\inria\phpMawTba.arff")

# Convertir a DataFrame de pandas
adult_census = pd.DataFrame(data)
adult_census.head(1)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25.00,b'Private',226802.00,b'11th',7.00,b'Never-married',b'Machine-op-inspct',b'Own-child',b'Black',b'Male',0.00,0.00,40.00,b'United-States',b'<=50K'


In [8]:
target_name = "class"
numerical_columns = ["age", "capital-gain", "capital-loss", "hours-per-week"]

target = adult_census[target_name]
target = target.str.decode('utf-8')  # Decodificar bytes a string
data = adult_census[numerical_columns]

data.head()

,age,capital-gain,capital-loss,hours-per-week
0,25.00,0.00,0.00,40.00
1,38.00,0.00,0.00,50.00
2,28.00,0.00,0.00,40.00
3,44.00,7688.00,0.00,40.00
4,18.00,0.00,0.00,30.00


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

model = Pipeline(
    steps=[
        ("preprocessor", StandardScaler()),
        ("classifier", LogisticRegression())
    ]
)

In [11]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target)
scores = cv_results['test_score']

print(
    "Acurracy socre via cross-validation: \n"
    f"{scores.mean():.3f} ± {scores.std():.3f}"
)

Acurracy socre via cross-validation: 
0.800 ± 0.003


We can also change the hyperparameter of a model after it has been created
with the `set_params` method, which is available for all scikit-learn
estimators. For example, we can set `C=1e-3`, fit and evaluate the model: